# Tool example

In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import PromptTemplate

llm = ChatVertexAI(model="gemini-2.0-flash-001")

Let's take a look the example of enhancing an LLM with tools. We will mock tools responses for now, and demonstrate how tool calling works in general. Let's instruct LLM either to use a tool or to reply to the user:

In [ ]:
question = "how old is the US president?"

raw_prompt_template = (
  "You have access to search engine that provides you an "
  "information about fresh events and news given the query. "
  "Given the question, decide whether you need an additional "
  "information from the search engine (reply with 'SEARCH: "
   "<generated query>' or you know enough to answer the user "
   "then reply with 'RESPONSE <final response>').\n"
   "Do not make any assumptions on recent events or things that can change."
   "Now, act to answer a user question:\n{QUESTION}"
)
prompt_template = PromptTemplate.from_template(raw_prompt_template)

Now let's call an LLM with a simple question that can be answered based on its internal knowledge:

In [ ]:
question1 = "What is the capital of Germany?"
result = (prompt_template | llm).invoke(question1)
print(result.content)

RESPONSE: Berlin



If we change the question, we can see how an LLM has decided to use a search tool and generated a search query to be used with it: 

In [ ]:
response = (prompt_template | llm).invoke(question)
print(response.content)

SEARCH: current age of the US president



We can make the next step, and give a history of previous intercations (including a response from the tool). Please, note that we haven't implemented any tools yet, we'll do it in the next sections:

In [ ]:
query = "age of current US president"
search_result = (
    "Donald Trump › Age 78 years June 14, 1946\n"
    "Donald Trump 45th and 47th U.S. President Donald John Trump is an American "
    "politician, media personality, and businessman who has served as the 47th "
    "president of the United States since January 20, 2025. A member of the "
    "Republican Party, he previously served as the 45th president from 2017 to 2021. Wikipedia"
)

raw_prompt_template = (
  "You have access to search engine that provides you an "
  "information about fresh events and news given the query. "
  "Given the question, decide whether you need an additional "
  "information from the search engine (reply with 'SEARCH: "
   "<generated query>' or you know enough to answer the user "
   "then reply with 'RESPONSE <final response>').\n"
   "Today is {date}."
   "Now, act to answer a user question and "
   "take into account your previous actions:\n"
   "HUMAN: {question}\n"
   "AI: SEARCH: {query}\n"
   "RESPONSE FROM SEARCH: {search_result}\n"
)
prompt_template = PromptTemplate.from_template(raw_prompt_template)

result = (prompt_template | llm).invoke({"question": question, "query": query, "search_result": search_result, "date": "Feb 2025"})
print(result.content)

RESPONSE: The current US president, Donald Trump, is 78 years old.



We can see that an LLM can decide to call a tool as the second step if the previous interaction hasn't provided a clear answer:

In [ ]:
query = "current US president"
search_result = (
    "Donald Trump 45th and 47th U.S."
)

raw_prompt_template = (
  "You have access to search engine that provides you an "
  "information about fresh events and news given the query. "
  "Given the question, decide whether you need an additional "
  "information from the search engine (reply with 'SEARCH: "
   "<generated query>' or you know enough to answer the user "
   "then reply with 'RESPONSE <final response>').\n"
   "Today is {date}."
   "Now, act to answer a user question and "
   "take into account your previous actions:\n"
   "HUMAN: {question}\n"
   "AI: SEARCH: {query}\n"
   "RESPONSE FROM SEARCH: {search_result}\n"
)
prompt_template = PromptTemplate.from_template(raw_prompt_template)

result = (prompt_template | llm).invoke({"question": question, "query": query, "search_result": search_result, "date": "Feb 2025"})
print(result.content)

SEARCH: Donald Trump age

